In [1]:
# depndencies
import pandas as pd
import requests
from splinter import Browser
from bs4 import BeautifulSoup as bs
from pprint import pprint
import time
import re

# CANADIAN INDEX OF WELLBEING

In [14]:
# prepare chromedriver
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

# set browser with url of Canadian Index of Wellbeing
url1 = 'https://uwaterloo.ca/canadian-index-wellbeing/reports/2016-canadian-index-wellbeing-national-report/executive-summary'
browser.visit(url1)
time.sleep(5)

# set up parser
html = browser.html
soup = bs(html, 'lxml')

In [15]:
url = 'https://uwaterloo.ca'

In [16]:
# get images
image1_url = url + soup.find_all('img')[0]['src']
image2 = soup.find_all('img')[1]['src']
image2_url = url + image2
image1_url

'https://uwaterloo.ca/canadian-index-wellbeing/sites/ca.canadian-index-wellbeing/files/resize/uploads/images/c011676-ciw-nationalreport-ciwvsgdp-750x500.jpg'

In [17]:
url2 = 'https://uwaterloo.ca/canadian-index-wellbeing/what-we-do/framework'
browser.visit(url2)
# get images
image3_url = url + soup.find_all('img')[0]['src']
image3_url

'https://uwaterloo.ca/canadian-index-wellbeing/sites/ca.canadian-index-wellbeing/files/resize/uploads/images/c011676-ciw-nationalreport-ciwvsgdp-750x500.jpg'

In [18]:
# Close the browser after scraping
browser.quit()

## Get main table

In [19]:
# use pandas to read the tables
tables = pd.read_html(url1)

In [20]:
t1=tables[0].copy()
t1 = t1.drop(t1.index[[10,11]])
t1['Year'] = t1['Year'].astype(int)
t1['Year1']=t1['Year']+1
t1["GDP percapita increase"]= t1["GDP per capita"].str.split(" ", n = 1, expand = True)[1].astype(float)
t1["CIW increase"]= t1["CIW"].str.split(" ", n = 1, expand = True)[1].astype(float)
t1=t1.drop(t1.columns[[0,1,2]], axis=1).set_index('Year1')
t1

,GDP percapita increase,CIW increase
Year1,,
1995,1.73,-0.73
1997,5.53,-1.23
1999,13.65,-0.50
2001,18.36,1.06
2003,21.59,3.80
2005,26.80,5.77
2007,30.36,8.39
2009,25.25,8.50
2011,34.05,5.91


In [21]:
t0=tables[0].copy()
t0 = t0.drop(t0.index[11])
t0['Year'] = t0['Year'].astype(int)
t0["GDP percapita increase"]= t0["GDP per capita"].str.split(" ", n = 1, expand = True)[0].astype(float)
t0["CIW increase"]= t0["CIW"].str.split(" ", n = 1, expand = True)[0].astype(float)
t0 = t0.drop(t0.columns[[1,2]], axis=1).set_index('Year')
t0

,GDP percapita increase,CIW increase
Year,,
1994,0.00,0.00
1996,2.22,-1.41
1998,8.62,-1.06
2000,18.20,-0.29
2002,20.42,2.35
2004,24.10,4.56
2006,29.09,7.21
2008,29.87,8.84
2010,30.87,6.43


In [22]:
vsGDP_Table = pd.concat([t0, t1]). sort_index()
vsGDP_Table.tail(1)

,GDP percapita increase,CIW increase
2014,37.99,9.93


In [23]:
CIW_Table=tables[1].copy()
CIW_Table.drop(CIW_Table.index[21], inplace=True)
CIW_Table['Year'] = CIW_Table['Year'].astype(int)
CIW_Table = CIW_Table.rename(columns={'Community Vitality': 'com_vty', 'Democratic Engagement': 'dem_egmt','Education': 'edu',
                          'Environment': 'env','Healthy Populations': 'hlty_pop', 'Leisure and Culture': 'leisure',
                          'Living Standards': 'liv_std', 'Time Use': 'time_use',}).set_index('Year')
CIW_Table.tail(1)

,CIW,com_vty,dem_egmt,edu,env,hlty_pop,leisure,liv_std,time_use
Year,,,,,,,,,
2014,9.9,14.8,13.0,32.8,-2.9,16.2,-9.3,11.9,3.0


In [24]:
#pd.concat([df1, df4], axis=1, sort=False)
JoinTable=pd.concat([vsGDP_Table,CIW_Table], axis=1, sort=False)
JoinTable.tail(1)

,GDP percapita increase,CIW increase,CIW,com_vty,dem_egmt,edu,env,hlty_pop,leisure,liv_std,time_use
2014,37.99,9.93,9.9,14.8,13.0,32.8,-2.9,16.2,-9.3,11.9,3.0


# Get Feature Tables

In [27]:
def table_maker_function(feature):
    url_a = f'https://uwaterloo.ca/canadian-index-wellbeing/reports/2016-canadian-index-wellbeing-national-report/{feature}'
    table_a = pd.read_html(url_a)[0].set_index('Year')
    url_b = f'https://uwaterloo.ca/canadian-index-wellbeing/reports/2016-canadian-index-wellbeing-national-report/{feature}-domain-raw-data'
    table_b = pd.read_html(url_b)[0].set_index('Year')
    Table = pd.concat([table_a,table_b], axis=1, sort=False)
    Table = Table.drop(Table.columns[[0,1]], axis=1)
    return Table

In [28]:
table_education = table_maker_function('education')
table_education.tail(1)

,Education,Percentage of children aged 0 to 5 years for whom there is a regulated centre-based childcare space,Average amount of time spent in talk-based activities with children aged 0 to 14 years,Average expenditure per public school student (2013$),Ratio of students to educators in public schools,Average annual Canadian undergraduate tuition fees (2015$),Percentage of Canadians 20 to 24 years of age in labour force completing high school,Percentage of 25 to 64 year olds in population with a university degree,Percentage of adults aged 25 and older participating in education related activities
Year,,,,,,,,,
2014,32.8,24.1,34.2,13.58,11.8,5998,89.3,28.5,5.7


In [29]:
table_healthy_populations = table_maker_function('healthy-populations')
table_healthy_populations.tail(1)

,Healthy Populations,Life expectancy at birth in years,Percentage of population that rates their overall health as very good or excellent,Percentage of population that rates their mental health as very good or excellent,Percentage of population with an absence of health or activity-based limitations,Percentage of daily or occasional smokers among teens aged 12 to 19 years,Percentage of population with self-reported diabetes,Percentage of population getting influenza immunization in past year,Percentage of Canadians with a regular medical doctor
Year,,,,,,,,,
2014,16.2,82.1,59.0,71.1,78.5,7.7,6.7,32.5,85.1


In [30]:
table_community_vitality = table_maker_function('community-vitality')
table_community_vitality.tail(1)

,Community Vitality,Percentage of population that reports very or somewhat strong sense of belonging to community,Percentage of population with 5 or more close friends,Percentage of population that feels safe walking alone after dark,Crime Severity Index,Percentage of population experiencing discrimination in past 5 years based on ethno-cultural characteristics,Percentage of population that believes that most or many people can be trusted,"Percentage of population reporting unpaid, formal volunteering for groups or organizations",Percentage of population that provides unpaid help to others on their own
Year,,,,,,,,,
2014,14.8,66.4,52.5,78.7,66.7,8.0,54.7,49.1,81.4


In [31]:
table_democratic_engagement = table_maker_function('democratic-engagement')
table_democratic_engagement.tail(1)

,Democratic Engagement,Percentage of voter turnout at federal elections,Ratio of registered to eligible voters,Gap in percentage turnout between older and younger voters,Percentage of women in federal Parliament,Percentage of Members of Parliament's office budget devoted to sending communications to constituents,"Percentage of population that volunteers for a law, advocacy, or political group",Percentage of population that is very of fairly satisfied with way democracy works in Canada,Percentage of population with a great deal or quite a lot of confidence in federal Parliament
Year,,,,,,,,,
2014,13.0,66.3,0.88,25.4,25.7,5.87,1.9,65.8,35.5


In [32]:
table_living_standards = table_maker_function('living-standards')
table_living_standards.tail(1)

,Living Standards,After-tax median income of economic family (2013$),Percentage of persons living in poverty based on low income cut-off (LICO),Gini coefficient (income gap),Percentage of households that are moderately or severely food insecure,Housing affordability based on Shelter Consumption Affordability Ratio (SCAR),Percentage of labour force employed,Percentage of labour force in long-term unemployment,CIBC index of employment quality (1994 Q1=100)
Year,,,,,,,,,
2014,11.9,72100,9.7,0.32,7.7,39.2,61.4,12.4,98.3


In [33]:
table_time_use = table_maker_function('time-use')
table_time_use.tail(1)

,Time Use,Percentage of Canadians 25 to 64 years of age working over 50 hours per week at main job,"Percentage of labour force working under 30 hours per week, not by choice","Percentage of labour force with regular, weekday work hours",Percentage of individuals working for pay with flexible work hours,Mean workday commute time for individuals working for pay,Percentage of Canadians who report 7 to 9 hours of good quality essential sleep,Average daily amount of time with friends (minutes per day),Percentage of 15 to 64 year olds reporting high levels of time pressure
Year,,,,,,,,,
2014,3.0,8.7,5.3,66.5,45.2,53.9,34.7,81.1,16.4


In [34]:
table_environment = table_maker_function('environment')
table_environment.tail(1)

,Environment,Ecological Footprint,Absolute GHG emissions (megatonnes of CO2 per year),Ground level ozone (population weighted in parts per billion),Primary energy production (petajoules),Viable Metal Reserves Index,"Residential energy use (terajoules per 1,000 dwellings)",Total farm land (hectares),Annual water yield in Southern Canada (km3)
Year,,,,,,,,,
2014,-2.9,8.284,732,32.8,11189704,0.59,100.02,63148311,1293.3


In [35]:
table_leisure_culture = table_maker_function('leisure-and-culture')
table_leisure_culture.tail(1)

,Leisure and Culture,Average percentage of time spent on previous day in social leisure activities,Average percentage of time spent on previous day in arts and culture activities,Average monthly frequency of participation in physical activity lasting over 15 minutes,Average attendance per performance at all performing arts performances,Average number of hours volunteering for culture and/or recreation organizations,Average visitation per site past year to National Parks and National Historic Sites,Average number of nights away on vacation trips to destinations at least 80km from home,Expenditures on all culture and recreation as a percentage of total household expenditures
Year,,,,,,,,,
2014,-9.3,13.17,3.93,27.56,328.44,34.3,194482,2.96,4.76
